# QaaS demo with Qibo and Qiboconnection


### Create a bit flip circuit with Qibo and Qiboconnection

Initially: 

$$q_0 = \ket{0} =
\begin{bmatrix}
1 \\
0
\end{bmatrix}

\\
q_1 = \ket{0} = \begin{bmatrix}
1 \\
0
\end{bmatrix}
\\
$$

Therefore, the initial state is always: 
$$\ket{\psi} = \ket{00} = \ket{0}\otimes\ket{0}= \begin{bmatrix}
1\\
0
\end{bmatrix}\otimes \begin{bmatrix}
1\\
0
\end{bmatrix} = \begin{bmatrix} 1\begin{bmatrix} 1 \\ 0 \end{bmatrix} \\ 0\begin{bmatrix} 1 \\ 0\end{bmatrix}\end{bmatrix}= \begin{bmatrix}
1 \\
0 \\
0 \\
0
\end{bmatrix}$$

Apply $\sigma_x$ to $q_1$:
$$\sigma_{x} \ket{0} = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}\begin{bmatrix}
1 \\
0
\end{bmatrix}

=
\begin{bmatrix}
0 \\
1
\end{bmatrix}=\ket{1}
\\$$
Hence, the final state will always be: 

$$q_0 = \ket{0} =
\begin{bmatrix}
1 \\
0
\end{bmatrix}

\\
q_1 = \ket{1} = \begin{bmatrix}
0 \\
1
\end{bmatrix}
\\
$$
Which can be represented as: 
$$\ket{01} = \begin{bmatrix}
1 \\
0
\end{bmatrix}\otimes\begin{bmatrix}
0 \\
1
\end{bmatrix} = \begin{bmatrix}
0 \\
1 \\
0 \\
0
\end{bmatrix}$$

In [1]:
from qibo.models import Circuit
from qibo.gates import X,M,H,CNOT
import qibo

In [2]:
bit_flip_circuit=Circuit(nqubits=1)
bit_flip_circuit.add(X(0))
bit_flip_circuit.add(M(0))  # you need to measure always, otherwise the execution will fail!

MeasurementResult(qubits=(0,), nshots=0)

In [3]:
bit_flip_circuit.draw()

'q0: ─X─M─'

In [4]:
qibo.set_backend('numpy')

[Qibo 0.1.12.dev0|INFO|2023-11-11 08:17:31]: Using numpy backend on /CPU:0


In [6]:
result = bit_flip_circuit(nshots=10)
result,result.frequencies()

((1+0j)|1>, Counter({'1': 10}))

### Connect to the API


In [7]:
import os 
from qiboconnection.api import API

In [8]:
os.environ["QIBOCONNECTION_ENVIRONMENT"] = "development" # target the development server!
api = API.login(username='pyday23-user',api_key='pyday23-apikey')

### Select device and execute circuit


In [9]:
api.list_devices()

<Devices[1]:
{
  "device_id": 1,
  "device_name": "dev_sim",
  "status": "online",
  "availability": "available",
  "characteristics": {
    "type": "simulator",
    "cpu": null,
    "gpu": null,
    "os": null,
    "kernel": null,
    "ram": null
  }
}

In [10]:
api.select_device_id(device_id=1) # select the simulator, otherwise the job will fail

In [11]:
api.execute(circuit=bit_flip_circuit, nshots=10)

[29252]

In [12]:
job_id=29252

### Retrieve job information, including results

In [13]:
api.get_job(job_id=job_id)

Your job with id 29252 is completed.


JobData(description=[<qibo.models.circuit.Circuit object at 0x7f2f19c70b50>], device_id=1, execution_type='qibo', job_id=29252, job_type='circuit', number_shots=10, queue_position=0, result=[{'probabilities': [0.0, 1.0], 'frequencies': {'1': 10}}], status='completed', user_id=16)

In [14]:
api.get_job(job_id=job_id).description[0].draw()

Your job with id 29252 is completed.


'q0: ─X─M─'

In [15]:
api.get_job(job_id=job_id).result

Your job with id 29252 is completed.


[{'probabilities': [0.0, 1.0], 'frequencies': {'1': 10}}]

## Apply Hadamard Gate with Qiboconnection $\rightarrow$ Slido question


In [16]:
hadamard_circuit = Circuit(1)
hadamard_circuit.add(H(0))
hadamard_circuit.add(M(0))

MeasurementResult(qubits=(0,), nshots=0)

In [150]:
# Draw the circuit
hadamard_circuit.draw()

'q0: ─H─M─'

In [17]:
job_id = api.execute(circuit=hadamard_circuit, nshots=100)[0]

In [152]:
api.get_job(job_id).result

Your job with id 29127 is completed.


[{'probabilities': [0.4999999999999999, 0.4999999999999999],
  'frequencies': {'0': 51, '1': 49}}]

## Random Number Generator with Qibo

In [27]:
# Execute the circuit 1 time
result = hadamard_circuit(nshots=1)

# Print the measurement result which is a random number 0 or 1
print(result.frequencies())

Counter({'0': 1})


In [28]:
def random_number_of_n_bits(n):
    measurements = [list(hadamard_circuit(nshots=1).frequencies().keys())[0] for _ in range(n)]
    string_binary = "".join(measurements)
    integer = int(string_binary, 2)
    return integer

In [34]:
random_number_of_n_bits(20)

562554

## Creating a Bell State with Qiboconnection

$$\ket{\psi} = \frac{\ket{00}+\ket{11}}{\sqrt{2}}$$

In [146]:
bell_state_circuit = Circuit(2)
bell_state_circuit.add(H(0))
bell_state_circuit.add(CNOT(0,1))
bell_state_circuit.add(M(0,1))

MeasurementResult(qubits=(0, 1), nshots=0)

In [147]:
job_id = api.execute(circuit=bell_state_circuit, nshots=100)[0]

In [148]:
api.get_job(job_id).result

Your job with id 29126 is completed.


[{'probabilities': [0.4999999999999999, 0.0, 0.0, 0.4999999999999999],
  'frequencies': {'00': 57, '11': 43}}]

## Slido Questions!!!!